# Comparação Intertextual usando Modelos NLP
#### *Luís F. Laguardia* - Visualização da Informação
Nesse trabalho, experimentamos comparar textos de diferentes tipos e estilos de modo quantitativo, usando modelos de processamento de lingaguem natural, ou NLP. Depois de gerarmos os modelos em si (em outro notebook) criamos uma função de pontuação para analisar a similaridade entre um fragmento de texto qualquer e os modelos carregados. Isso nos possibilitou analisar tanto trechos de textos independentes quanto fazer comparações de textos intermodelos e, com isso, criamos visualizações interessantes a partir dessas comparações.

Por fim, já com foco nas aplicações dessa experiência e em criar mais visualizações atrativas, fizemos plots a respeito da taxa de convergência desses modelos para a resposta final, ou quantas palavras são necessárias em um fragmento de texto para encontrar qual o modelo treinado mais se aproxima do seu conteúdo. 

In [1]:
import pickle

# carrega os modelos já treinados
m_tweets = pickle.load(open('models/tweets.pkl', 'rb'))
m_legenda = pickle.load(open('models/legenda.pkl', 'rb'))
m_livro = pickle.load(open('models/livro.pkl', 'rb'))
m_lyrics = pickle.load(open('models/lyrics.pkl', 'rb'))
m_wikipedia = pickle.load(open('models/wikipedia.pkl', 'rb'))
m_receitas = pickle.load(open('models/receitas.pkl', 'rb'))

model_names = ['tweets', 'legenda', 'receitas', 'letras', 'wikipedia', 'livro'] # nomes dos modelos
min_score = [-15.79218, -18.87291, -17.28105, -17.24841, -19.99931, -19.55732] # p(1) para cada modelo


# 1. Comparando um fragmento de texto com todos os modelos

In [2]:
# função que retorna a pontuação de um fragmento de texto para cada modelo
def get_scores(text):
    score = []

    for i, model in enumerate([m_tweets, m_legenda, m_receitas, m_lyrics, m_wikipedia, m_livro]):
        mean_score = 0
        for word in text.split():
            mean_score += 1 - (model.logscore(word) / min_score[i])
        mean_score /= len(text.split())
        mean_score = round(mean_score, 4)
        score.append(mean_score)
    return score

In [3]:
# função que cria o plot em heatmap baseada nas pontuações
import plotly.express as px

def plot_heatmap(text):
    perplexity = get_scores(text)
    fig = px.imshow([perplexity],
    text_auto=True,
    color_continuous_scale='YlGnBu')

    title = ""
    if len(text) < 40:
        title = f'Pontuação para <i>"{text}"</i>.'
    else:
        title = 'Pontuação para o texto:'


    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [0, 1, 2, 3, 4, 5],
            ticktext = model_names,
            title = title,
        ),
        yaxis = dict(
            visible = False
        ),
        
    )
    fig.update_xaxes(side="top")
    fig.show()

In [4]:
# Teste o modelo com um fragmento de texto qualquer
plot_heatmap("Três colheres de chá")

# 2. Comparando os textos dos modelos entre eles mesmos

In [23]:
# função que retorna uma amostra de tamanho n de uma das fontes de texto escolhidas
from nltk import word_tokenize
import re
import glob
import random

def get_sample(font: str, n=200):
    PASTA = font
    PATH = f'{PASTA}/*.txt'
    files = glob.glob(PATH)

    full_texts = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            tokens = word_tokenize(text)
            full_texts += tokens

    max_start = len(full_texts) - n
    start = random.randrange(0, max_start)
    
    return full_texts[start:start+n]

In [6]:
# célula que cria o heatmap para a comparação entre as fontes de texto e modelos
import plotly.express as px
results = []

for font in ["tweets", "legenda", "receitas", "lyrics", "wikipedia", "livro"]:
    sample = get_sample(font, 3000)
    line = get_scores(' '.join(sample))
    results.append(line)

fig = px.imshow(results,
color_continuous_scale='YlGnBu')
fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [0, 1, 2, 3, 4, 5],
            ticktext = [f"<b>{model}</b>" for model in model_names],
            side = "top",
            title = "Modelos"
        ),
        yaxis = dict(
            tickmode = 'array',
            tickvals = [0, 1, 2, 3, 4, 5],
            ticktext = model_names,
        ),
    )
    

# 3. Visualização da Convergência

In [109]:
# célula que gera a base de dados para o plot 3d de convergência
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
m = [np.array([1, 2, 3, 4, 5, 6])*i for i in range(6)]
m = pd.DataFrame(m, columns=model_names, index=model_names)

result = []
sample_sizes = [1, 5, 50, 100, 1000, 3000]
for sample_size in sample_sizes:
    sample_matrix = []
    for font in ["tweets", "legenda", "receitas", "lyrics", "wikipedia", "livro"]:
        sample = get_sample(font, sample_size)
        line = get_scores(' '.join(sample))
        sample_matrix.append(line)
    result.append(sample_matrix)

# convert result into data structure
data = []
for z in range(6):
    for y in range(6):
        for x in range(6):
            data.append(dict(
                x=x,
                y=y,
                sample_size=z,
                score=result[z][y][x]
            )) 
df = pd.DataFrame(data)

In [111]:
# célula que cria o plot 3d de convergência
fig = go.Figure()
for i in range(6):
    df_sec = df[df['sample_size'] == i]

    fig.add_trace(go.Scatter3d(x = df_sec['x'], y = df_sec['y'], z = df_sec['sample_size'],
    mode='markers',
    showlegend=True,
    name=f'{sample_sizes[i]} amostra(s)',
    legendgroup=f'group {i}',
    hovertext=df_sec['score'],
    marker=dict(
        size=12,
        color=df_sec['score'],
        colorscale='YlGnBu',              
        cmax=0.6,
    )
    ))

fig.update_layout(
    scene = dict(
            xaxis = dict(
                tickmode = 'array',
                tickvals = [0, 1, 2, 3, 4, 5],
                ticktext = [f"<b>{model}</b>" for model in model_names],
                title = "Modelos",
            ),
            yaxis = dict(
                tickmode = 'array',
                tickvals = [0, 1, 2, 3, 4, 5],
                ticktext = model_names,
                title = "Fontes"
            ),
            zaxis = dict(
                tickmode = 'array',
                tickvals = [0, 1, 2, 3, 4, 5],
                ticktext = sample_sizes,
                title = "Tamanho da amostra"
            ),
        ),
    height=600,
    width=800,
)

# 3.1: visualização da convergência para um modelo em específico (Wikipédia)

In [127]:
# célula que mostra a convergência da pontuação para uma amostra no modelo da Wikipédia

# Amostra de texto não usado no treinamento
text_sample = """
Em geometria analítica, um vetor é uma classe de equipolência de segmentos de reta orientados, que possuem todos a mesma intensidade (também designada por norma ou módulo), mesma direção e mesmo sentido.[1] Em alguns dos casos, a expressão vetor espacial também é utilizada.[carece de fontes]
Neste contexto, um vetor {\displaystyle \mathbf {a} }\mathbf{a} pode ser representado por qualquer segmento de reta orientado, que seja membro da classe deste vetor (isto é, por qualquer segmento de reta orientado que possua o mesmo módulo, mesma direção e mesmo sentido de qualquer outro segmento da referida classe). Se o segmento {\displaystyle {\overline {AB}}}\overline{AB} (segmento de reta orientado do ponto A para o ponto B) for um representante do vetor {\displaystyle \mathbf {a} ,}\mathbf{a}, então podemos dizer que o vetor {\displaystyle \mathbf {a} }\mathbf{a} é igual ao vetor {\displaystyle {\overrightarrow {AB}}.}\overrightarrow{AB}.
Podemos ainda representar um vetor como um número complexo na forma {\displaystyle z=a+bi}z=a+bi, onde {\displaystyle a}a representa a abcissa e {\displaystyle b}b representa a ordenada desse vetor.[2]
Um vetor é definido como sendo uma classe de equipolência de segmentos de reta orientados de {\displaystyle \mathbb {V} ^{n}}\mathbb{V}^n[3], em que {\displaystyle \mathbb {V} ^{n}}\mathbb{V}^n representa um espaço vetorial de n dimensões. Assim sendo, em um espaço vetorial de 3 dimensões ({\displaystyle \mathbb {V} ^{3}}\mathbb{V}^3), cada vetor será dotado de três coordenadas, comumente denominadas x, y e z.
Os vetores desempenham um papel importante na física: velocidade e aceleração de um objeto e as forças que agem sobre ele são descritas por vetores. Os componentes de um vetor dependem do sistema de coordenadas usado para descrevê-lo. Outros objetos usados para descrever quantidades físicas são os pseudovetores e tensores.
Os vetores têm aplicação em várias áreas do conhecimento, tanto técnico quanto científico, como física, engenharia e economia, por exemplo, sendo os elementos a partir dos quais se constrói o Cálculo Vetorial.
"""
text = text_sample.split()

scores = []
for end in range(1, len(text)):
    sample = text[0:end]
    mean_score = 0
    for word in sample:
        mean_score += m_wikipedia.logscore(word)
    mean_score /= len(sample)
    scores.append(1 - mean_score/-19.99931)

fig = px.area(x=range(1, len(text)), y=scores, title="Score por tamanho da amostra")
fig.update_layout(
    xaxis_range = [0, len(text)],
    yaxis_range = [0.3, 0.6]
)